### ollama聊天模板

In [ ]:
# pip install ollama
import ollama
import re
messages =[]
#加上system很快爆内存
# messages = [{'role':'system','content':'你将用尽可能简洁的语言与用户聊天'}]
while (True):
  question = input()
  if len(messages)>=8:
    for i in range(2):
      messages.pop(1)
  if len(question)==0:
    break
  #gemma:2b, qwen:4b-chat-v1.5-q6_K，qwen:7b-chat, qwen:14b-chat
  #gemma:7b-instruct-q8_0
  messages.append({'role': 'user', 'content': question})
  # messages[{'role': 'user', 'content': question}]
  print('用户：', question)
  print('Bot：', end='')
  stream = ollama.chat(
      # model='qwen:4b-chat-v1.5-q6_K',#可以长多轮对话，速度也挺快
      model='qwen:7b-chat-v1.5-q5_0',#跟14b一样对话轮次少
      # model='qwen:14b-chat',
      messages=messages,
      stream=True,
      #option中的参数含义：https://github.com/ggerganov/llama.cpp/tree/master/examples/main#number-of-tokens-to-predict
      options={
      'num_predict': 128,#生成的最大tokens数,default:128
      'temperature': 0.7,#default:0.9
      # 'top_p': 0.9,
      # 'top_k':20#defalut:40
      }
  )
  outword=""
  num=0
  for chunk in stream:
    result = chunk['message']['content']
    result = re.sub('\n+','\n', result) 
    outword+=result
    print(result, end='', flush=True)
    num+=1
  # print(stream['message']['content'],flush=True)
  messages.append({'role': 'assistant','content':outword})
  print()

### 常规输入模型转为Promot格式--ollama似乎不需要

In [ ]:
def translate(input_list, is_translate=True, is_input=True):
    '''
    将input_list转为Promot格式
    is_translate:是否进行转换
    '''
    if not is_translate:
        return(input_list)
    input_list_copy = input_list.copy()
    start = "<|im_start|>"
    end= "<|im_end|>"
    for index in range(len(input_list_copy)):
        if type(input_list_copy[index]) == dict:
            input_list_copy[index]=(f"{start}{input_list_copy[index]['role']}\n{input_list_copy[index]['content']}{end}\n")
    if is_input:
        input_list_copy.append(f"{start}{'assistant'}\n")
    return(input_list_copy)

messages = [{'role':'system','content':'你被邀请来陪用户聊天'}]
messages.append({'role': 'user', 'content': "你能跟我聊天吗"})
print(''.join(translate(messages)))
print('*'*40)
outword='''对不起，我不知道你的名字是什么。'''
messages.append({'role': 'assistant','content':outword})
print(''.join(translate(messages, is_input=False)))

### ai回复并发声

In [ ]:

def text_voice(engine, rate=-1, volume=-1, voice=-1):
    '''
    三个参数：
    rate:速率
    volume:音量，【0,1】
    voice:声音，0为男声，1为女声
    '''
    if rate!=-1:
        engine.setProperty('rate', rate)
    if volume!=-1:
        engine.setProperty('volume',volume)
    if voice!=-1:
        voices = engine.getProperty('voices')       
        engine.setProperty('voice', voices[voice].id)
    return engine

#pyttsx3无法说话
import pyttsx3
engine = pyttsx3.init()
engine = text_voice(engine, rate=130)
(r,ve,vs) = (engine.getProperty('rate'),engine.getProperty('volume'), engine.getProperty('voices'))
print('目前语速是：',r,'\t','语音音量是:',ve)


import ollama
# import speech
stream = ollama.chat(
    model='gemma:2b',
    messages=[{'role': 'user', 'content': 'do you have a happy memorary'}],
    stream=True,
)
outword2=""
outword=""
outword3=""

for chunk in stream:
    outword3 = chunk['message']['content']
    outword2 += outword3
    outword += outword3
    #语音播放
    if outword3 in {',','.','。','，','!','！', '?', '？',':', '：'}:
        outword = outword.replace('\n','').replace('*','')
        # speech.say(outword)
        engine.say(outword)
        
        outword=""
    engine.runAndWait()
    #打印对话内容
    print(outword3, end='', flush=True)